In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.sparse.linalg import eigs
import random

# ======= CONFIGURATION ========
INPUT_FILE =   # Update with your input file
SHEET_NAME =   # Sheet containing the aggressive matrix
OUTPUT_FILE =  # Update with output path

# Elo rating parameters
K_FACTOR = 32           # Choose using previous studies or prior knowledge of system
INITIAL_RATING = 1000   # Can be arbitrary
RANDOM_REPEATS = 100    # Number of random shuffles for Elo


# Load the aggressive interaction matrix
df_matrix = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME, index_col=0)
bird_names = df_matrix.index.tolist()
n_birds = len(bird_names)

# Convert to numeric matrix
W = df_matrix.values.astype(float)

# CALCULATE DAVID'S SCORE & RANK 
# Create the probability matrix P
P = np.zeros_like(W)
for i in range(n_birds):
    for j in range(n_birds):
        if i != j:
            total = W[i, j] + W[j, i]+ + 1e-5
            P[i, j] = W[i, j] / total if total > 0 else 0.5

# Calculate weighted wins (w) and losses (l)
w = P.sum(axis=1)
l = P.sum(axis=0)

# Calculate second-order terms
w2 = P @ w
l2 = P.T @ l

# Calculate David's Scores
DS = (w + w2) - (l + l2)

# Convert to David's Rank (1 = highest score)
DR = np.argsort(np.argsort(-DS)) + 1

# CALCULATE RANDOMIZED ELO RATING 
# Create event list: (winner, loser) pairs
events = []
for i in range(n_birds):
    for j in range(n_birds):
        if i != j and W[i, j] > 0:
            # Add each win as a separate event
            events.extend([(i, j)] * int(W[i, j]))

# Initialize storage for Elo ratings
elo_storage = np.zeros((RANDOM_REPEATS, n_birds))

# Run multiple randomized sequences
for rep in range(RANDOM_REPEATS):
    ratings = np.full(n_birds, INITIAL_RATING)
    random.shuffle(events)  # Shuffle event order
    
    for winner, loser in events:
        # Calculate expected win probability
        rating_diff = ratings[loser] - ratings[winner]
        expected = 1 / (1 + 10 ** (rating_diff / 400))
        
        # Update ratings
        ratings[winner] += K_FACTOR * (1 - expected)
        ratings[loser] -= K_FACTOR * (1 - expected)
    
    elo_storage[rep] = ratings

# Average ratings across repetitions
avg_elo = np.mean(elo_storage, axis=0)

# Convert to Elo Rank (1 = highest rating)
elo_rank = np.argsort(np.argsort(-avg_elo)) + 1

# ======= 3. EIGENVECTOR CENTRALITY ========
# Ensure matrix is irreducible (add small epsilon to avoid zeros)
epsilon = 1e-6
W_irreducible = W + epsilon

# Calculate eigenvector centrality
eigenvalues, eigenvectors = eigs(W_irreducible, k=1, which='LM')
centrality = np.abs(eigenvectors.real.flatten())

# Convert to Eigenvector Rank (1 = highest centrality)
eigen_rank = np.argsort(np.argsort(-centrality)) + 1


# Calculate Spearman correlations
ds_elo_corr, _ = spearmanr(DR, elo_rank)
ds_eigen_corr, _ = spearmanr(DR, eigen_rank)
elo_eigen_corr, _ = spearmanr(elo_rank, eigen_rank)

# Create correlation matrix
correlation_matrix = pd.DataFrame({
    'David\'s Rank': [1.0, ds_elo_corr, ds_eigen_corr],
    'Elo Rank': [ds_elo_corr, 1.0, elo_eigen_corr],
    'Eigen Rank': [ds_eigen_corr, elo_eigen_corr, 1.0]
}, index=['David\'s Rank', 'Elo Rank', 'Eigen Rank'])


# Create DataFrames for results
ds_df = pd.DataFrame({
    'Bird': bird_names,
    'David\'s Score': DS,
    'David\'s Rank': DR
}).sort_values('David\'s Rank')

elo_df = pd.DataFrame({
    'Bird': bird_names,
    'Elo Rating': avg_elo,
    'Elo Rank': elo_rank
}).sort_values('Elo Rank')

eigen_df = pd.DataFrame({
    'Bird': bird_names,
    'Eigenvector Centrality': centrality,
    'Eigen Rank': eigen_rank
}).sort_values('Eigen Rank')

# Write to Excel
with pd.ExcelWriter(OUTPUT_FILE) as writer:
    ds_df.to_excel(writer, sheet_name='David\'s Score', index=False)
    elo_df.to_excel(writer, sheet_name='Elo Rating', index=False)
    eigen_df.to_excel(writer, sheet_name='Eigenvector Centrality', index=False)
    correlation_matrix.to_excel(writer, sheet_name='Rank Correlations')
    
    # Save the win matrix for reference
    df_matrix.to_excel(writer, sheet_name='Win Matrix')

print(f"Analysis complete! Results saved to {OUTPUT_FILE}")
print("\nSpearman Rank Correlations:")
print(correlation_matrix)